# Intro to Decks: OT2
### In this tutorial, you will see how to instantiate an OT2 Deck, how to add labware, and how to move liquids around.

First we will import `LiquidHandler`, a backend called `ChatterBoxBackend` that prints the text
output of our commands, a class `Visualizer` that provides a visualization of the robot deck as we
run commands, and a class `OTDeck` that will represent the deck of an OpenTrons OT2, one of
the most widely used liquid handling robots. 

Make sure to also `import opentrons` !

### Imports

In [1]:
import pylabrobot

In [2]:
pylabrobot.__file__

'/mnt/d/Chory Lab/PyLabRobot/pylabrobot/__init__.py'

In [4]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import ChatterBoxBackend
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.resources.opentrons import OTDeck

from pylabrobot.resources.opentrons.load import *
from pylabrobot.resources.opentrons.plates import *

from pylabrobot.resources import set_tip_tracking, set_volume_tracking
set_tip_tracking(True), set_volume_tracking(True)

import opentrons
import time

### Setting up the Deck and Visualizer

First, we will create an instance of the `LiquidHandler` class. This may take some time to set up, so we run the `setup()` function with the `await` keyword.

In [5]:
lh = LiquidHandler(backend=ChatterBoxBackend(), deck=OTDeck())

await lh.setup()

Setting up the robot.
Resource deck was assigned to the robot.
Resource trash_container was assigned to the robot.


After initializing our `LiquidHandler`, we want to create an instance of a `Visualizer`. This will allow you to see the Deck and follow your protocol in real time. You can see how tips and liquids move as you run commands. Make sure to open the `Visualizer` in another window.

In [6]:
vis = Visualizer(resource=lh)
await vis.setup()

Websocket server started at http://127.0.0.1:2121
File server started at http://127.0.0.1:1337 . Open this URL in your browser.


### Adding Labware to the Deck

Now, we are ready to add some labware to the deck. **PyLabRobot** has many different labware items already defined. Only import the ones that you need for your protocol. A full list of labware can be found in `PyLabRobot\Resources\opentrons`. You can also create custom labware, but that is out of scope for this tutorial.

Let's begin by importing a `TubeRack`, a `TipRack`, and a `Plate`.
<details>
    <summary>Definitions of Labware:</summary>
    
* **TubeRack** = Used to hold various tubes, commonly the 2mL Eppendorfs.

* **TipRack** = Labware that holds pipette tips.

* **Plate** = Well-Plate that one can add liquids to.
</details>

In [7]:
from pylabrobot.resources import (
    opentrons_24_tuberack_eppendorf_2ml_safelock_snapcap,
    opentrons_96_tiprack_300ul,
    corning_96_wellplate_360ul_flat
)

**Note:** The OT2 has **11 spots** for labware. It also has a built in trash can for discarding tips.

One thing to keep in mind when designing a protocol is `layout efficiency`. The more separated labware is on the deck, the longer your protocol will be because the pipette has to travel farther.

In general, you want to keep your tip racks in the back, your stocks in the middle, and then finally your plates in the front. This allows for `linear movement`.

For example, the arm grabs a **tip from slot 7**, aspirates **stock from slot 4**, and then finally **dispenses in slot 1**. Reducing the travel time of the pipette will decrease the runtime of your protocol.

To place labware on the Deck, call `assign_child_at_slot()`. Pass in the labware you want to place along with the slot you want to place it in.

In [8]:
# When you instantiate a labware, give it a name that will show when you
# mouse over it in the visualizer.
tip_rack = opentrons_96_tiprack_300ul("tip_rack")

lh.deck.assign_child_at_slot(tip_rack, 7)

Resource tip_rack was assigned to the robot.


In [9]:
# Stock Solution Tube Rack
tube_rack = opentrons_24_tuberack_eppendorf_2ml_safelock_snapcap("tube_rack")

lh.deck.assign_child_at_slot(tube_rack, 4)

Resource tube_rack was assigned to the robot.


In [10]:
# Plate for Sample Preparation
plate = corning_96_wellplate_360ul_flat("prep_plate")

lh.deck.assign_child_at_slot(plate, 1)

Resource prep_plate was assigned to the robot.


In [11]:
print(lh.deck.summary())


Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: Empty       | 11: Empty       | 12: trash_co... |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: tip_rack    |  8: Empty       |  9: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: tube_rack   |  5: Empty       |  6: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: prep_plate  |  2: Empty       |  3: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+



### Adding Liquids to the Deck

Let's add some liquids to our `tube_rack`. We can add up to the `max_volume` of the tube. If you go over this number, **PyLabRobot** will throw an error. We shall add 1000µL of 4 different dyes to the first column of our `tube_rack`. This corresponds to wells *A1, B1, C1, and D1*.

To iterate over locations on labware, we use the `traverse()` function. This produces a generator object that we use the `next` keyword on to yield our desired wells.

`traverse()` takes in two arguments: **batch_size** is the amount of wells to return, and **direction** is how to iterate over the wells. In our case, we use `"down"` to return the wells column wise.

<details>
    <summary>More info on <b>direction:</summary>

* `"down"`, `"snake_down"`, `"right"`, and `"snake_right"` start at the top left item **(A1)**.
        
* `"up"` and `"snake_up"` start at the bottom left **(H1)**.
    
* `"left"` and `"snake_left"` start at the top right **(A12)**.

* The `snake` directions alternate between going in the given direction and going in the opposite direction. For example, `"snake_down"` will go from A1 to H1, then H2 to A2, then A3 to H3, etc.
</details>

In [12]:
first_col_tubes = next(tube_rack.traverse(batch_size=4, direction='down'))
first_col_tubes

[Tube(name=tube_rack_A1, location=(018.210, 075.430, 041.270), size_x=6.223, size_y=6.223, size_z=39.1, category=tube),
 Tube(name=tube_rack_B1, location=(018.210, 056.150, 041.270), size_x=6.223, size_y=6.223, size_z=39.1, category=tube),
 Tube(name=tube_rack_C1, location=(018.210, 036.870, 041.270), size_x=6.223, size_y=6.223, size_z=39.1, category=tube),
 Tube(name=tube_rack_D1, location=(018.210, 017.590, 041.270), size_x=6.223, size_y=6.223, size_z=39.1, category=tube)]

Use the `tracker.set_liquids()` function to put liquid in a `tube`. The `tracker` class contains all of the methods associated with keeping record of how much/what kind of liquid is in a given container.

Pass in a string for **"Liquid_Type"** and a number for **Volume** to this function.

Let's add our dyes to the tubes in `first_col_tubes`.

In [13]:
for i in range(len(first_col_tubes)):
    # [(liquid, volume)]
    first_col_tubes[i].tracker.set_liquids([(f"Dye_{i}", 2000)])

Here's a Utility function for printing all of the filled spots of a `TubeRack`.

In [14]:
def print_filled_spots_of_tubeRack(tube_rack):

    all_tubes = tube_rack.get_all_children()

    all_empty = True
    
    for tube in all_tubes:
    
        liquid = tube.tracker.liquids

        if liquid != []:
            print(f"Spot {tube.name.split('_')[-1]} contains:")

            name = liquid[0][0]
            vol = liquid[0][1]
        
            print(f"{vol}uL of {name}")

            all_empty = False

    if all_empty:
        print("Entire rack is empty!")

In [15]:
print_filled_spots_of_tubeRack(tube_rack)

Spot A1 contains:
2000uL of Dye_0
Spot B1 contains:
2000uL of Dye_1
Spot C1 contains:
2000uL of Dye_2
Spot D1 contains:
2000uL of Dye_3


In [16]:
print_filled_spots_of_tubeRack(plate)

Entire rack is empty!


### Moving Liquids from Point A to Point B

Now that we've added some dyes to our tube rack, let's use the robot to move some liquid to our `prep_plate`. The first step of a liquid transfer is acquiring a tip.

You can acquire a tip by calling `lh.pick_up_tips()`, and passing in the `TipSpots` of the tips you want to retrieve. `TipSpots` are indexed the same way that wells are.

When a tip is picked up from a `TipRack`, it's location will turn white on the visualizer. To see what tips are currently on the robot, call `lh.head`. This returns a dictionary where the keys are the indices of the different channels of the main pipettor, and where the values are instances of the `TipTracker` class, allowing you to get information about how tips move on and off of a given channel.

If you want to reset the state of tips on a robot, call `lh.return_tips()`. This function will automatically return the tips to their original locations.

In [76]:
await lh.return_tips()

Dropping tips [Drop(resource=TipSpot(name=tip_rack_C3, location=(032.380, 056.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Drop(resource=TipSpot(name=tip_rack_E7, location=(068.380, 038.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Drop(resource=TipSpot(name=tip_rack_A1, location=(014.380, 074.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Drop(resource=TipSpot(name=tip_rack_D3, location=(032.380, 047.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47))].


Let's try picking up 8 tips along the diagonal.

In [17]:
await lh.pick_up_tips(tip_rack["A1", "B2", "C3", "D4", "E5", "F6", "G7", "H8"])

Picking up tips [Pickup(resource=TipSpot(name=tip_rack_A1, location=(014.380, 074.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_B2, location=(023.380, 065.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_C3, location=(032.380, 056.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_D4, location=(041.380, 047.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(

Rather than use `lh.return_tips()` all of the time, you can also call `lh.drop_tips()` and pass in specifically where to place the tip, and what channel's tip to drop.

In [18]:
await lh.drop_tips(tip_spots = tip_rack["A1", "B2", "C3", "D4", "E5", "F6", "G7", "H8"],
                   use_channels = [0,1,2,3,4,5,6,7])

Dropping tips [Drop(resource=TipSpot(name=tip_rack_A1, location=(014.380, 074.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Drop(resource=TipSpot(name=tip_rack_B2, location=(023.380, 065.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Drop(resource=TipSpot(name=tip_rack_C3, location=(032.380, 056.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Drop(resource=TipSpot(name=tip_rack_D4, location=(041.380, 047.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Drop(resource=Tip

The order in which you pass in the `tip_spots` and the `use_channels` lists will determine which channel gets which tip. Take a look!

In [19]:
await lh.pick_up_tips(tip_spots = tip_rack["A1", 'C3', "E7"], use_channels = [2,0,1])

Picking up tips [Pickup(resource=TipSpot(name=tip_rack_A1, location=(014.380, 074.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_C3, location=(032.380, 056.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_E7, location=(068.380, 038.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47))].


In [32]:
lh.head

{0: TipTracker(Channel 0, is_disabled=False, has_tip=False tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47), pending_tip=None),
 1: TipTracker(Channel 1, is_disabled=False, has_tip=True tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47), pending_tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)),
 2: TipTracker(Channel 2, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 3: TipTracker(Channel 3, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 4: TipTracker(Channel 4, is_disabled=False, has_tip=True tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47), pending_tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)),
 5: TipTracker(Channel 5, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 6: TipTracker(Channel 6, is_disabled=False, has_tip=False tip=N

Here's another utility function that will print the status of tips on the pipetter. If there is a tip on a channel, this function will output its origin.

In [21]:
def print_channels_tip_origin(lh):
    # Prints the origin location of all tips currently on the robot
    cur_pipetter = lh.head

    for channel in cur_pipetter:
        print(f"Channel {channel}:")

        tip_tracker = lh.head[channel]
        
        if tip_tracker.has_tip == True:
            print(tip_tracker.get_tip_origin())
        else:
            print("No tip present.")
        print()

In [41]:
def print_channel_status(lh):
    # Prints the status of liquids, if present, in each channel
    cur_pipetter = lh.head

    for channel in cur_pipetter:
        print(f"Channel {channel}:")

        tip_tracker = lh.head[channel]
        
        if tip_tracker.has_tip == True:
            tip = tip_tracker.get_tip()
            print(tip.tracker.liquids)
        else:
            print("No tip present.")
        print()

In [33]:
print_channels_tip_origin(lh)

Channel 0:
No tip present.

Channel 1:
TipSpot(name=tip_rack_E7, location=(068.380, 038.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot)

Channel 2:
No tip present.

Channel 3:
No tip present.

Channel 4:
TipSpot(name=tip_rack_D3, location=(032.380, 047.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot)

Channel 5:
No tip present.

Channel 6:
No tip present.

Channel 7:
No tip present.



In [45]:
print_channel_status(lh)

Channel 0:
No tip present.

Channel 1:
[]

Channel 2:
No tip present.

Channel 3:
No tip present.

Channel 4:
[]

Channel 5:
No tip present.

Channel 6:
No tip present.

Channel 7:
No tip present.



In [54]:
lh.head[1].get_tip().tracker.pending_liquids

[]

You also don't need to specify channels in order. If you wanted to skip channel 3, we can do so by just passing in the next channel index and skipping 3.

In [23]:
await lh.pick_up_tips(tip_rack["D3"], use_channels = [4])

Picking up tips [Pickup(resource=TipSpot(name=tip_rack_D3, location=(032.380, 047.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47))].


In [24]:
await lh.drop_tips(tip_spots = tip_rack["E7"], use_channels = [2])

Dropping tips [Drop(resource=TipSpot(name=tip_rack_E7, location=(068.380, 038.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47))].


If you want to check if a spot in the tip rack has a tip, call `TipSpot.has_tip()`. This could be useful if you are writing a script with many operations and want to automatically calculate where you should grab your next tip from.

In [25]:
tip_rack["A1"][0].has_tip()

False

In [44]:
await lh.aspirate(tube_rack["A1"], vols=[200])

Aspirating [Aspiration(resource=Tube(name=tube_rack_A1, location=(018.210, 075.430, 041.270), size_x=6.223, size_y=6.223, size_z=39.1, category=tube), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47), volume=200, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[('Dye_0', 200)])].


In [26]:
await lh.aspirate(tube_rack["A1"], vols=[200])
time.sleep(2)

await lh.dispense(plate["A1"], vols=[200])
#plate["A1"][0].tracker.commit()
time.sleep(2)

await lh.return_tips()

Aspirating [Aspiration(resource=Tube(name=tube_rack_A1, location=(018.210, 075.430, 041.270), size_x=6.223, size_y=6.223, size_z=39.1, category=tube), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47), volume=200, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[('Dye_0', 200)])].
Dispensing [Dispense(resource=Well(name=prep_plate_A1, location=(014.380, 074.240, 003.550), size_x=4.851, size_y=4.851, size_z=10.67, category=well), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47), volume=200, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[('Dye_0', 200)])].


HasTipError: Tip spot already has a tip.

In [55]:
print_filled_spots_of_tubeRack(plate)

Spot A1 contains:
200uL of Dye_0


In [27]:
await lh.pick_up_tips(tip_rack["A1"])
time.sleep(1)

await lh.aspirate(plate["A1"], vols=[200])
time.sleep(1)

await lh.dispense(tube_rack["A1"], vols=[200])
time.sleep(1)

await lh.return_tips()

Picking up tips [Pickup(resource=TipSpot(name=tip_rack_A1, location=(014.380, 074.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47))].
Aspirating [Aspiration(resource=Well(name=prep_plate_A1, location=(014.380, 074.240, 003.550), size_x=4.851, size_y=4.851, size_z=10.67, category=well), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47), volume=100, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[('Dye_0', 100)])].
Dispensing [Dispense(resource=Tube(name=tube_rack_A1, location=(018.210, 075.430, 041.270), size_x=6.223, size_y=6.223, size_z=39.1, category=tube), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47), volume=100, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[('Dye_0', 100)])].
Dropping tips [Drop(re